# &#x2699; Efficient random walks

## Bare-bone random walks

We have seen [earlier](rw) a simple implementation of random walks in Python.

In [22]:
import numpy as np


def naive_random_walker(N, M, delta=1.0, dims=1):
    """
    Simulates a random walk. Parameters:
    - N: number of steps
    - M: number fo walkers
    - delta: size of one step (default=1)
    - dims : number of dimensions (default=1)
    """

    # The initial position is the origin
    position = np.zeros((N, M, dims))
    # Random walks
    for i in range(1, N):
        for w in range(M):
            dr = delta * np.random.choice([-1, 1], size=dims)
            position[i, w] = position[i - 1, w] + dr
    return position


To inspect the performance of the code, we can use the **profiler** included in jupyter notebooks. 

This is invoked via a so-called *magic* command `prun`, prepended in front of teh function. We also use the optional argument `-s cumulative` to sort the output in terms of the cumulative time spent in different parts of the code.

In [33]:
%prun -s cumulative naive_random_walker(100,1000)

          1881005 function calls in 4.041 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.041    4.041 {built-in method builtins.exec}
        1    0.000    0.000    4.041    4.041 <string>:1(<module>)
        1    0.620    0.620    4.040    4.040 2149943443.py:4(naive_random_walker)
    99000    0.972    0.000    3.421    0.000 {method 'choice' of 'numpy.random.mtrand.RandomState' objects}
   198000    0.222    0.000    2.449    0.000 <__array_function__ internals>:177(prod)
   198000    0.145    0.000    2.189    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   198000    0.240    0.000    2.044    0.000 fromnumeric.py:2955(prod)
   198000    0.528    0.000    1.804    0.000 fromnumeric.py:69(_wrapreduction)
   198000    0.836    0.000    0.836    0.000 {method 'reduce' of 'numpy.ufunc' objects}
   198000    0.232    0.000    0.232    0.000 {built-in method

The first three lines are not interesting as they refer to the function object itself.
The fourth item is immediately interesting as it refers to `method 'choice' of 'numpy.random.mtrand.RandomState'`. Clearly, the way we are generating the random steps is time-consuming, as we are picking from a Python list of two elements.

Such high-level calculations are typically  not performing well. We can therefore improve the performance of the code by considering an alternative way to generate lists of arrays such as `[-1,1,..]` in $d$ dimensions.

The simplest option is to use another function from `numpy` leveraging a paritcle to spin mapping that is common in physics.

Suppose that $n$ is a variable describing the occupation number of a site on a lattice, so that $n=0,1$. If we want to map empty sites to down spins and filled sites to up spins we can perform a chage of variable

$$s = 2n-1$$

so that $s=-1,1$ for $n=0,1$.

The same idea can be employed to our code, where we can replace the call of `np.random.choice` with `np.random.randint(0,2)*2-1`

In [44]:
def improved_random_walker(N, M, delta=1.0, dims=1):
    """
    Simulates a random walk. Parameters:
    - N: number of steps
    - M: number fo walkers
    - delta: size of one step (default=1)
    - dims : number of dimensions (default=1)
    """

    # The initial position is the origin
    position = np.zeros((N, M, dims))
    # Random walks
    for i in range(1, N):
        for w in range(M):
            dr = delta * np.random.randint(0, 2, size=dims) * 2 - 1
            position[i, w] = position[i - 1, w] + dr
    return position


We can compare the two using the `%%time` magic command

In [40]:
%%time 
pos = naive_random_walker(100,1000)

CPU times: user 2.82 s, sys: 34.1 ms, total: 2.85 s
Wall time: 2.86 s


In [45]:
%%time 
pos = improved_random_walker(100,1000)

CPU times: user 2.06 s, sys: 17.3 ms, total: 2.07 s
Wall time: 2.09 s


The `naive` code is about 40% percent slower than the `improved` code.

Can we do better?

The code entails two `for` loops. In Python, for loops are slow and nested loops are particularly so.

The simplest way to accelerate the code is to leverage the strength of just in time compilers like [`numba`](https://numba.pydata.org). 

You can install numba via 

```
pip install numba
```

`numba` accelerates the code through a clever translation of the Python code into `C/C++` code which is compiled on-the fly. This means that the first execution of the code can be slow (as pre-compilation occurs), but the second and following executions will benefit from the `C` translation.

For the translation to take place, `numba` needs to correctly infer the variable types used in the code. 

A black-box usage of numba is possible and is straighforward. It requires the addition of a **decorator** `@numba.njit` in fornt of the function that we want to accelerate.

In [14]:
import numba as nb


@nb.njit
def numba_random_walker(N, M, delta=1.0, dims=1):
    """
    Simulates a random walk. Parameters:
    - N: number of steps
    - M: number fo walkers
    - delta: size of one step (default=1)
    - dims : number of dimensions (default=1)
    """

    # The initial position is the origin
    position = np.zeros((N, M, dims))
    # Random walks
    for i in range(1, N):
        for w in range(M):
            dr = delta * np.random.randint(0, 2, size=dims) * 2 - 1
            position[i, w] = position[i - 1, w] + dr
    return position


The first execution of the code is already very fast

In [46]:
%%time
walks = numba_random_walker(100,1000)

CPU times: user 35.4 ms, sys: 9.43 ms, total: 44.8 ms
Wall time: 51.8 ms


the second execution is even faster

In [47]:
%%time
walks = numba_random_walker(100,1000)

CPU times: user 39 ms, sys: 3.38 ms, total: 42.4 ms
Wall time: 45.3 ms


Wanna go faster and do not mind writing a little Fortran code yourself? Try `f2py-jit`!

In [10]:
from f2py_jit import jit

code = """
subroutine rw(position, delta)
  real(8), intent(inout) :: position(:, :, :)
  real(8), intent(in) :: delta
  real(8) :: dr(size(position, 3))
  do i = 1, size(position, 1)
    do j = 1, size(position, 2)
      call random_number(dr)
      dr = delta * (floor(dr*2) - 1)
      position(i, j, :) = position(i-1, j, :) + dr
     end do
  end do
end subroutine rw
"""

def f2py_random_walker(N, M, dims=1):
    import numpy
    position = numpy.zeros((N, M, dims), order='F')
    f90 = jit(code)
    f90.rw(position, 1.0)

In [11]:
%%time
walks = f2py_random_walker(100, 1000)    

CPU times: user 3.58 ms, sys: 3.73 ms, total: 7.32 ms
Wall time: 884 ms


In [12]:
%%time
walks = f2py_random_walker(100, 1000) 

CPU times: user 3.58 ms, sys: 0 ns, total: 3.58 ms
Wall time: 3.14 ms


This is appreciably faster than `numba`. Note that, like with numba, the first time the execution is slower.

How does this compare with Python's numerical library, `numpy`?

The random walk sampler can indeed be written in a single lines of code using `numpy`:

In [75]:
def numpy_random_walker(N, M, delta=1.0, dims=1):
    """
    Simulates a random walk. Parameters:
    - N: number of steps
    - M: number fo walkers
    - delta: size of one step (default=1)
    - dims : number of dimensions (default=1)
    """
    return np.cumsum(
        delta * np.random.randint(0, 2, size=(N, M, dims)) * 2 - 1.0, axis=0
    )


We exploit the fact that `numpy` can perform cumulative sums rapidly over the array, and that one can specify the dimension along which to perform the sum with the parameter `axis`. In our case, the zeroth axis is the one that corresponds to time.

In [81]:
%%time
walks = numpy_random_walker(100,1000)

CPU times: user 2.14 ms, sys: 1.25 ms, total: 3.39 ms
Wall time: 1.73 ms


`numpy` provides an even faster solution, with a further speedup of a factor around 20, for a total speedup of about 1200 times with respect to the original naive code. 

In summary, we started with a naive Python code for a simulation of a d-dimensional random walk and accelerated it around 60 times with a few tweaks. But leveraging the vectorised operations of the numericla library provided an even larger benefit, allowign us to sample easily large numbers of walks.   

Notice, also, that all the different versions of the code store the entire statistics of walks throught time, which becomes memory intensive for large numbers of time steps, number of walks or high dimensions.

### Summary of the algorithms

Here below you can find the three main versions of the bare-bond random walk functions discussed above.

`````{tab-set}
````{tab-item} Naive Python 
```python
def naive_random_walker(N, M, delta=1.0, dims=1):
    """
    Simulates a random walk. Parameters:
    - N: number of steps
    - M: number fo walkers
    - delta: size of one step (default=1)
    - dims : number of dimensions (default=1)
    """

    # The initial position is the origin
    position = np.zeros((N, M, dims))
    # Random walks
    for i in range(1, N):
        for w in range(M):
            dr = delta * np.random.choice([-1, 1], size=dims)
            position[i, w] = position[i - 1, w] + dr
    return position
```
````
````{tab-item} Numba
```python
@nb.njit
def numba_random_walker(N, M, delta=1.0, dims=1):
    """
    Simulates a random walk. Parameters:
    - N: number of steps
    - M: number fo walkers
    - delta: size of one step (default=1)
    - dims : number of dimensions (default=1)
    """

    # The initial position is the origin
    position = np.zeros((N, M, dims))
    # Random walks
    for i in range(1, N):
        for w in range(M):
            dr = delta * np.random.randint(0, 2, size=dims) * 2 - 1
            position[i, w] = position[i - 1, w] + dr
    return position
```
````
````{tab-item} Numpy
```python
def numpy_random_walker(N, M, delta=1.0, dims=1):
    """
    Simulates a random walk. Parameters:
    - N: number of steps
    - M: number fo walkers
    - delta: size of one step (default=1)
    - dims : number of dimensions (default=1)
    """
    return np.cumsum(np.random.randint(0, 2, size=(N, M, dims)) * 2 - 1.0, axis=0)

```
````


`````

## Particle simulations and random walks

andom walks provide a mathematical and numerical framework to model a variety of problems, such as the movement of molecules in a liquid, fluctuations of stock prices, the path of an animal looking for food, and the spread of diseases in a population.

In this book we focus on liquids. It is pedagogical to explore how we can embed our knowledge of random walks in a framework for particle based simulations of liquids that we will use through the text.

We leverage the python model `atooms`. This provides a simplified interface to paritcle based simulations with various dynamics. Interestingly, it also allows us to provide arbritrary dynamics to simulate a given model. 

Here we test this idea in the case of random walks.


### Brief introduction to `atooms`

`atooms` is object oriented: it contains objects with attributes and methods that allow us to access and modify the properties of the objects themselves and that also allow the objects to communicate with one another.

`atooms` has one key object: it is the `Simulation` object, which contains all the interesting properties of a particles-based simulation, including the spatial domain of the simulation (i.e. the simulation *box*)

In [ ]:
import numpy as np


class RandomWalk(object):
    def __init__(self, system, delta=1.0):
        self.system = system
        self.delta = delta

    def run(self, steps):
        for i in range(steps):
            for p in self.system.particle:
                dr = numpy.array([random() - 0.5, random() - 0.5, random() - 0.5])
                dr *= self.delta
                p.position += dr
